In [1]:
import os,sys
import librosa
import IPython
import random
import math
import librosa
import time
import logging
import numpy as np
import soundfile as sf
from pesq import pesq
import torch
import  torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import json
import warnings
warnings.filterwarnings("ignore")
from Ushape_model_without_ref import CNN_BLSTM, Ushape_Backbone, Ushape_Att_Backbone, Ushape_Att_Backbone_With_Ave, Dense_Ushape_CNN_Backbone, Stoi_net, Dense_Ushape_CNN_Backbone_With_Ave, Blstm_With_Ave

def create_logger(title="", dir_path='.'):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    timestamp = time.time()  # 当前时间戳
    strtime = time.strftime("%m-%d %H-%M-%S", time.localtime(timestamp))
    logfile = '%s.txt' % (title+strtime)
    logfile_path = os.path.join(dir_path, logfile)
    fh = logging.FileHandler(logfile_path, mode='a')
    ch = logging.StreamHandler()
    # formatter = logging.Formatter("%(asctime)s -  %(message)s")
    # fh.setFormatter(formatter)
    # ch.setFormatter(formatter)
    logger.addHandler(fh)
    logger.addHandler(ch)
    return logger


In [2]:
num_epoch = 2

model_layer = 1 #dense: 1-774 2-1170 3-1782    cnn:1-516   # 更新：dense:1-645 2-975 3-1485 cnn:1-516 2-520 3-528 blstm:1-257
neurons_num = 257

use_dataset = 3
rand_seed = 9987
evalu_target = 'pesq'
select_model = "stoinet" # "cnn" or "dense_cnn"or "mosnet" or "stoinet" or "cnn_with_ave" or "dense_cnn_with_ave" or "blstm_with_ave"

if use_dataset == 1:
    wav_path = '/data/gwx/musan/speech_librivox_slice_8s'
    file_dir = '/data/gwx/musan/speech_librivox_slice_8s_add_noise'
    ref_npy_path = '/data/gwx/musan/speech_librivox_slice_8s_npy'

elif use_dataset == 2:
    wav_path = '/data/gwx/musan/speech_librivox_slice_8s'
    file_dir = '/data/gwx/musan/speech_artificial_8s_noise_dataset/'
    ref_npy_path = '/data/gwx/musan/speech_librivox_slice_8s_npy'
    
elif use_dataset == 3:
    wav_path = '/data/gwx/musan/Version_2_speech_librivox_slice_8s'
    file_dir = '/data/gwx/musan/Version_2_speech_artificial_8s_noise_dataset'
    ref_npy_path = '/data/gwx/musan/Version_2_speech_librivox_slice_8s_npy'
    
log_dir = './output_without_reference'


if select_model is 'cnn':
    log_sub_dir_name = 'CNN%.1d_LSTM_FC-' % model_layer + "data_%1d" % use_dataset
elif select_model is 'mosnet':
    log_sub_dir_name = "MosNet-dataset_%1d" % use_dataset
elif select_model is 'stoinet':
    log_sub_dir_name = "StoiNet_With_Ave_Without0-dataset_%1d" % use_dataset
elif select_model is 'dense_cnn_with_ave':
    log_sub_dir_name = 'Dense_CNN%.1d_LSTM_FC_With_Ave_Without0-' % model_layer + "dataset_%1d" % use_dataset
elif select_model is 'blstm_with_ave':
    log_sub_dir_name = 'BLSTM_FC_With_Ave_Without0-' + "dataset_%1d" % use_dataset


log_model_save_path = os.path.join(log_dir, evalu_target, log_sub_dir_name)
if not os.path.exists(log_model_save_path):
    os.makedirs(log_model_save_path)
    
logger = create_logger('rand_seed_%d-'%rand_seed, log_model_save_path)    
model_ckpt_save_path = os.path.join(log_model_save_path, "best_epoch.ckpt")

class speech_Dataset(Dataset):
    def __init__(self, file_dir, file_list, evalu_target, wav_path, ref_npy_path):
        self.file_dir = file_dir
        self.file_list = file_list
        self.wav_path = wav_path
        self.ref_npy_path = ref_npy_path
        self.target = evalu_target
        

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self,idx):
#         file_index = idx // 1000
#         frame_index = idx % 1000 + 1
#         frame_name = 'iter_%.4d' % frame_index
#         file_data = np.load(os.path.join(self.file_dir, self.file_list[file_index]), allow_pickle=True).item()
        file_path = self.file_list[idx]
        file_data = np.load(file_path, allow_pickle=True).item()
        ref_name = file_path.split('/')[-2]
        
        if not os.path.exists(os.path.join(self.ref_npy_path, ref_name+'.npy')):
            ref_dict = {}
            ref_data, _ = librosa.load(os.path.join(self.wav_path, ref_name+'.wav'), 16000)
            ref_stft = librosa.stft(ref_data, n_fft=512, hop_length=256)
            ref_dict["raw_data"] = ref_data
            ref_dict["stft_transf"] = ref_stft
#             print(self.ref_npy_path, ref_name)
            np.save(os.path.join(self.ref_npy_path, ref_name), ref_dict)
            ref_stft = np.abs(ref_stft)
        
        else:
            ref_dict = np.load(os.path.join(self.ref_npy_path, ref_name+'.npy'), allow_pickle=True).item()
            ref_data = ref_dict['raw_data']
            ref_stft = np.abs(ref_dict["stft_transf"])
            

        system_data = file_data['raw_data']
        frame_score = file_data['frame_score']
        
        if self.target is 'pesq':
            system_score = file_data['system_score']
#             print(system_score)
        elif self.target is 'stoi':
            system_score = file_data['stoi_system_score']
            
        system_stft = np.abs(file_data['stft_transf'])
#         system_stft = librosa.amplitude_to_db(np.abs(librosa.stft(system_data, n_fft=512, hop_length=256)), ref=np.max)

                                
        frame_score = frame_score.repeat(8, axis=0)
        frame_score.resize(1, system_stft.shape[1])
        frame_score = np.expand_dims(frame_score.repeat(system_stft.shape[0], axis=0), 0)
        system_stft = np.expand_dims(system_stft, 0)
        ref_stft = np.expand_dims(ref_stft, 0)
        frame_feature = np.concatenate((system_stft, ref_stft), axis=0)
        return frame_score, system_score, np.expand_dims(frame_feature[0,:,:], 0)

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)  # cpu
    torch.cuda.manual_seed_all(seed)  # gpu
    torch.backends.cudnn.deterministic = True  # consistent results on the cpu and gpu
    torch.backends.cudnn.benchmark = False

set_seed(rand_seed)

In [3]:
file_list = []

def print_list_dir(doc, path, suffix='.npy'):
    #遍历文件夹下面所有以某一后缀结尾的文件
    temp_list = os.listdir(path)
    for file in temp_list:
        temp_path = os.path.join(path, file)
        if os.path.isdir(temp_path):
            print_list_dir(doc, temp_path, suffix)
        elif temp_path.endswith(suffix):
            doc.append(temp_path)

if use_dataset == 1:
    sub_file_dir = os.listdir(file_dir)       
    for sub_dir in sub_file_dir:
        root_path = os.path.join(file_dir, sub_dir)
        file_temp = os.listdir(root_path)
        file_temp = [os.path.join(root_path, file_name) for file_name in file_temp]
        file_list.extend(file_temp)
        
    file_list.sort()
    random.shuffle(file_list)
    
    train_list = file_list[0:100000]
    test_list = file_list[100000: 110000]
    val_list = file_list[110000:]
elif use_dataset == 2:
    bg_gaus_datalist = []
    rand_gaus_1s_datalist = []
    file_gs_dir = 'guassian-bg_only/'
    file_1s_dir = 'guassian-rand_1s/'
    
    with open("dataset_gaussian_bg_only.txt", "r") as f:
        for line in f.readlines():
            data_path_pre = line.strip('\n')
            data_path = data_path_pre.replace("bg_guassian", "guassian-bg")
            bg_gaus_datalist.append(os.path.join(file_dir, file_gs_dir, data_path))
    
    with open("dataset_gaussian_rand_1s.txt", "r") as f:
        for line in f.readlines():
            data_path = line.strip('\n')
            rand_gaus_1s_datalist.append(os.path.join(file_dir, file_1s_dir, data_path))
            
elif use_dataset == 3:    
    bg_only_list = []
    path = "/data/gwx/musan/Version_2_speech_artificial_8s_noise_dataset/guassian-bg_only"
    print_list_dir(bg_only_list, path)

    rand_1s_list = []
    path = "/data/gwx/musan/Version_2_speech_artificial_8s_noise_dataset/guassian-rand_1s"
    print_list_dir(rand_1s_list, path)
            
#     file_list = bg_gaus_datalist + rand_gaus_1s_datalist
    file_list = bg_only_list + rand_1s_list
    file_list.sort()
    random.shuffle(file_list)
    
    train_list = file_list[0:160000]
    test_list = file_list[160000: 175000]
    val_list = file_list[175000:]

train_list.sort()
test_list.sort()
val_list.sort()
# selcet_list = []
# for file in file_list:
#     file_size = os.path.getsize(os.path.join(file_dir, file))
#     if file_size > 1000000:
#         selcet_list.append(file)
#         print(file)
# selcet_list.sort()
print(len(file_list))
print(file_list[:2])
            
# train_dataset = speech_Dataset(file_dir, train_list, evalu_target)
train_dataset = speech_Dataset('', train_list, evalu_target, wav_path, ref_npy_path)
train_loader = DataLoader(train_dataset,batch_size=1, num_workers=16, shuffle=True)

# val_dataset = speech_Dataset(file_dir, val_list, evalu_target)
val_dataset = speech_Dataset('', val_list, evalu_target, wav_path, ref_npy_path)
val_loader = DataLoader(val_dataset, batch_size=1, num_workers=8, shuffle=False)

# test_dataset = speech_Dataset(file_dir, test_list, evalu_target)
test_dataset = speech_Dataset('', test_list, evalu_target, wav_path, ref_npy_path)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=8, shuffle=False)

183380
['/data/gwx/musan/Version_2_speech_artificial_8s_noise_dataset/guassian-rand_1s/speech-slice-004884/guassian-rand_1s-0009-bg_snr_26db-snr_-10db.npy', '/data/gwx/musan/Version_2_speech_artificial_8s_noise_dataset/guassian-rand_1s/speech-slice-000097/guassian-rand_1s-0008-bg_snr_24db-snr_12db.npy']


In [4]:
# encoder_layer_1 = nn.TransformerEncoderLayer(d_model=64, nhead=1, batch_first=True, dim_feedforward=128)
# model = nn.TransformerEncoder(encoder_layer_1, num_layers=2)
# model = CNN_ATT_FC()

if select_model is 'cnn':
    model = Ushape_Att_Backbone(input_channels=1, out_channels=4, Ushape_layers=model_layer, att_dims=neurons_num)
elif select_model is 'mosnet':    
    model = CNN_BLSTM()
elif select_model is 'stoinet':    
    model = Stoi_net()
elif select_model is 'dense_cnn_with_ave': 
    model = Dense_Ushape_CNN_Backbone_With_Ave(input_channels=1, out_channels=4, Ushape_layers=model_layer, att_dims=neurons_num)
elif select_model is 'blstm_with_ave': 
    model = Blstm_With_Ave(input_channels=1, out_channels=4, Ushape_layers=model_layer, att_dims=neurons_num)

# pe = PositionalEncoding(64, 0, 10)
#output = model(src, n)
if torch.cuda.is_available():
    model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10, verbose=True)
# loss_fcn = nn.MSELoss()
loss_fcn1 = nn.MSELoss()
loss_fcn2 = nn.MSELoss()

In [ ]:
min_train_loss = 0.02 * 3 * 2
min_val_loss = 0.03 * 3 * 2

for iter_epoch in range(num_epoch):
    model.train()
#     loss_np = np.zeros(0)
    loss_np_500 = np.zeros(0)
    for num, data in enumerate(train_loader):
#         frame_score = data[0].unsqueeze(1).type(torch.float32)
#         frame_score = pe(frame_score)
#         print(frame_score[0,:,:])
#         frame_score, indices = torch.sort(frame_score)
        system_score = data[1].type(torch.float32)
#         print(system_score, type(system_score))
#         print(system_score < 4)
        if system_score == 0:
            print('pass 0')
            continue
        
        frame_feature = data[2].type(torch.float32)

        if torch.cuda.is_available():
#             frame_score = frame_score.cuda()
            system_score = system_score.cuda()
            frame_feature = frame_feature.cuda()
        
#         output = model(frame_feature)
        avg_score, frame_score = model(frame_feature)

#         loss = loss_fcn(output, system_score)
        loss = loss_fcn1(system_score, avg_score) + loss_fcn2(system_score, frame_score)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         loss_np = np.append(loss_np, loss.cpu().detach().item())
        loss_np_500 = np.append(loss_np_500, loss.cpu().detach().item())
        
    
            
#         print(loss.item())
        if num % 500 == 0:
            if num // 500 == 0:
                mean_loss = float("inf") 
                logger.info("iter: %d, loss: %f"%(num, loss.item()))
            
            if num // 500 >= 1:
#                 mean_loss = loss_np[(num//500-1)*500:num//500*500].mean()
                mean_loss = loss_np_500.mean()
                logger.info("iter: %d, loss: %f, mean loss: %f"%(num, loss.item(), mean_loss))
                loss_np_500 = np.zeros(0)
                
            if mean_loss < min_train_loss:
                model.eval()
                val_loss_list = np.zeros(0)
                for val_num, val_data in enumerate(val_loader):
                    val_system_score = val_data[1].type(torch.float32)
                    if val_system_score == 0:
                        print('pass 0')
                        continue

                    val_frame = val_data[2].type(torch.float32)

                    if torch.cuda.is_available():
                        val_system_score = val_system_score.cuda()
                        val_frame = val_frame.cuda()

#                         val_output = model(val_frame)
                    val_avg_score, val_frame_score = model(val_frame)

#                         val_loss = loss_fcn(val_output, val_system_score)
                    val_loss = loss_fcn1(val_system_score, val_avg_score) + loss_fcn2(val_system_score, val_frame_score)


                    val_loss_list = np.append(val_loss_list, val_loss.cpu().detach().item())

                if val_loss_list.mean() < min_val_loss:
                    min_train_loss = mean_loss.copy()
                    min_val_loss = val_loss_list.mean().copy()
                    torch.save(model.state_dict(), model_ckpt_save_path)
                logger.info("val in iter %d, epoch %d, mean loss is %.8f, min_val_loss is %.8f, min_train_loss is %.8f"
                            %(num, iter_epoch, val_loss_list.mean(), min_val_loss, min_train_loss))
                model.train()
#     break

iter: 0, loss: 4.652361


In [ ]:
# loss_list.mean()
# MSE = np.square(pred_list-gt_list).mean()
# print(loss_list.mean(), MSE) 

In [ ]:
model.eval()
pred_list = np.zeros(0)
gt_list = np.zeros(0)
loss_list = np.zeros(0)
model.load_state_dict(torch.load(model_ckpt_save_path))

for num, data in enumerate(test_loader):
#         frame_score = data[0].unsqueeze(1).type(torch.float32)
#         frame_score = pe(frame_score)
#         print(frame_score[0,:,:])
#         frame_score, indices = torch.sort(frame_score)
        system_score = data[1].type(torch.float32)
        if system_score == 0:
            print('pass 0')
            continue
        
        frame_feature = data[2].type(torch.float32)
        
        if torch.cuda.is_available():
#             frame_score = frame_score.cuda()
            system_score = system_score.cuda()
            frame_feature = frame_feature.cuda()
        
#         output = model(frame_feature)
        avg_score, frame_score = model(frame_feature)

        loss = loss_fcn1(frame_score, system_score)
#         loss = loss_fcn1(system_score, avg_score) + loss_fcn2(system_score, frame_score)

#         pred = output.cpu().detach().numpy()[0, 0]
        pred = avg_score.cpu().detach().numpy()[0, 0]
    
        gt = system_score.cpu().detach().numpy()[0]
        loss = loss.cpu().item()
        
        pred_list = np.append(pred_list, pred)
        gt_list = np.append(gt_list, gt)
        loss_list = np.append(loss_list, loss)
        logger.info("iter: %d, pred: %f, gt: %f, system loss: %f"%(num, pred, gt, loss))

In [ ]:
np.save(os.path.join(log_model_save_path, "prediction of testset"), pred_list)
np.save(os.path.join(log_model_save_path, "GT of testset"), gt_list)

In [ ]:
# pred_x = [0, 1, 2, 3, 4, 5, 6]#预测得分
# gt_y = [0, 1, 2, 3, 4, 5, 6]#真实得分
fig1 = plt.figure()
title_name = "Pred_GT-Relationship"
a1 = fig1.add_axes([0,0,1,1])

plt.title(title_name)
plt.grid(color = 'b', linestyle = '--', linewidth = 0.1)
plt.scatter(pred_list,gt_list, alpha=0.5)#预测得分和真实得分的关系图

a1.set_ylim(0,5)
a1.set_xlim(0,5)

plt.plot()
plt.show()

fig1.savefig(os.path.join(log_model_save_path, title_name+'.png'), dpi = 350, bbox_inches='tight')

In [ ]:
title_name = 'Testset-GT_score-Distribution'
fig2 = plt.figure()
plt.hist(gt_list,
        bins = 50,
        color = 'steelblue',
        edgecolor = 'black'
        )
plt.xlabel(title_name)
plt.ylabel('number')
plt.show()
fig2.savefig(os.path.join(log_model_save_path, title_name+'.png'), dpi = 350, bbox_inches='tight')

In [ ]:
title_name = 'Testset-Prediction_score-Distribution'
fig3 = plt.figure()
plt.hist(pred_list,
        bins = 50,
        color = 'steelblue',
        edgecolor = 'black'
        )
plt.xlabel('Pred score')
plt.ylabel('number')
plt.show()
fig3.savefig(os.path.join(log_model_save_path, title_name+'.png'), dpi = 350, bbox_inches='tight')

In [ ]:
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

MSE = np.square(pred_list-gt_list).mean()
print("MSE: ", MSE)

LCC = pearsonr(pred_list, gt_list)
print("LCC: ", LCC[0])

SRCC = spearmanr(pred_list, gt_list)
print("SRCC: ", SRCC[0])

indicators_name = 'result.txt'
with open(os.path.join(log_model_save_path, indicators_name), 'w') as writer:
    writer.write("MSE: %f\n" %MSE)
    writer.write("LCC: %f\n" %LCC[0])
    writer.write("SRCC: %f\n" %SRCC[0])